In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

def create_progress_evaluator_model():
    """
    Crea un modelo para evaluar el progreso del usuario y predecir
    la probabilidad de recordar una palabra.
    """
    # Definir el modelo
    model = Sequential([
    # Capa de entrada - 3 características
    Dense(16, activation='relu', input_shape=(3,)),
    Dropout(0.2), # Prevenir overfitting
    # Capa oculta
    Dense(8, activation='relu'),
    Dropout(0.1),
    # Capa de salida - probabilidad de recordar la palabra
    Dense(1, activation='sigmoid')
    ])
    # Compilar el modelo
    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )
    return model
# Crear el modelo
progress_model = create_progress_evaluator_model()

In [6]:
'''
def generate_synthetic_training_data(num_samples=1000):
    """
    Genera datos sintéticos para el entrenamiento inicial del modelo.
    """
    X = np.zeros((num_samples, 7))
    y = np.zeros((num_samples, 1))
    for i in range(num_samples):
    # Generar características aleatorias
    times_reviewed = np.random.randint(1, 10)
    times_correct = np.random.randint(0, times_reviewed + 1)
    times_incorrect = times_reviewed - times_correct
    days_since_last = np.random.randint(0, 30)
    avg_response_time = np.random.uniform(1.0, 10.0)
    word_difficulty = np.random.randint(1, 6)
    word_level_idx = np.random.randint(0, 4) # 0:A1, 1:A2, 2:B1, 3:B2
    # Probabilidad "real" (sintética) basada en reglas lógicas
    p_remember = 0.9 * (times_correct / max(1, times_reviewed)) - \
    0.05 * days_since_last - \
    0.1 * word_difficulty + \
    0.2 * (1.0 - min(1.0, avg_response_time / 10.0))
    # Ajustar a rango [0, 1]
    p_remember = max(0.01, min(0.99, p_remember))
    # Asignar valores
    X[i] = [times_reviewed, times_correct, times_incorrect,
    days_since_last, times_correct/max(1, times_reviewed),
    avg_response_time, word_difficulty]
    # El objetivo es 1 si se recuerda, 0 si no
    y[i] = 1 if np.random.random() < p_remember else 0
    return X, y
'''
import pandas as pd

df = pd.read_csv('duolingo.csv')

# Generar datos sintéticos
X_train, y_train = df.loc[:,("times_reviewed","times_correct","delta")],df.loc[:,"recall"] 
# Entrenamiento inicial
progress_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
# Guardar el modelo
progress_model.save('vocabulary_progress_model.h5')

Epoch 1/20
96134/96134 [==============================] - 91s 936us/step - loss: 0.2586 - accuracy: 0.8517 - val_loss: 0.2332 - val_accuracy: 0.8534
Epoch 2/20
96134/96134 [==============================] - 95s 990us/step - loss: 0.2390 - accuracy: 0.8540 - val_loss: 0.2339 - val_accuracy: 0.8506
Epoch 3/20
96134/96134 [==============================] - 87s 904us/step - loss: 0.2370 - accuracy: 0.8540 - val_loss: 0.2317 - val_accuracy: 0.8540
Epoch 4/20
96134/96134 [==============================] - 90s 939us/step - loss: 0.2362 - accuracy: 0.8541 - val_loss: 0.2330 - val_accuracy: 0.8532
Epoch 5/20
96134/96134 [==============================] - 83s 867us/step - loss: 0.2358 - accuracy: 0.8541 - val_loss: 0.2315 - val_accuracy: 0.8535
Epoch 6/20
96134/96134 [==============================] - 96s 1ms/step - loss: 0.2356 - accuracy: 0.8541 - val_loss: 0.2337 - val_accuracy: 0.8534
Epoch 7/20
96134/96134 [==============================] - 93s 971us/step - loss: 0.2354 - accuracy: 0.8541 -

In [7]:
# Estructura de datos de entrada

user_word_interaction = {
 "user_id": "user123",
 "word_id": "word456",
 "features": {
 "times_reviewed": 3, # Número de veces que ha visto la palabra
 "times_correct": 2, # Veces que respondió correctamente
 "delta": 2, # Días desde la última revisión
 }
}


In [10]:
'''
def determine_review_priority(user_words_data):
    """
    Determina qué palabras necesitan ser repasadas con mayor prioridad.
    Args:
    user_words_data: Lista de diccionarios con datos de interacción
    Returns:
    Lista de IDs de palabras ordenadas por prioridad de repaso
    """
    word_priorities = []
    for word_data in user_words_data:
    recall_prob = predict_recall_probability(word_data)
    # Calcular prioridad: palabras con baja probabilidad de ser recordadas
    # pero que han sido revisadas tienen alta prioridad
    priority_score = (1 - recall_prob) * (word_data["times_reviewed"] + 1)
    word_priorities.append({
    "word_id": word_data["word_id"],
    "recall_probability": recall_prob,
    "priority_score": priority_score
    })
    # Ordenar por prioridad (mayor primero)
    sorted_priorities = sorted(word_priorities,
    key=lambda x: x["priority_score"],
    reverse=True)
    return [item["word_id"] for item in sorted_priorities]

'''

def predict_recall_probability(user_word_data):
    """
    Predice la probabilidad de que un usuario recuerde una palabra
    basado en sus interacciones previas.
    Args:
    user_word_data: Dict con los datos de interacción usuario-palabra
    Returns:
    Probabilidad (0-1) de que el usuario recuerde la palabra
    """
    # Extraer características
    features = [
    user_word_data["times_reviewed"],
    user_word_data["times_correct"],
    user_word_data["delta"],
    ]
    # Convertir a array y dar forma adecuada
    features_array = np.array(features).reshape(1, -1)
    # Realizar predicción
    probability = progress_model.predict(features_array)[0][0]
    return float(probability)


predict_recall_probability(user_word_interaction["features"])



0.6727309823036194